In [227]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)
match_lvl_data = pd.read_csv(r"C:\Users\Chitwan\Downloads\match_level_scorecard.csv")
batsman_lvl_data = pd.read_csv(r"C:\Users\Chitwan\Downloads\batsman_level_scorecard.csv")
bowler_lvl_data = pd.read_csv(r"C:\Users\Chitwan\Downloads\bowler_level_scorecard.csv")
train_data = pd.read_csv(r"C:\Users\Chitwan\Downloads\train_data_with_samplefeatures.csv")
test_data = pd.read_csv(r"C:\Users\Chitwan\Downloads\test_data_with_samplefeatures.csv")

# Batsman_Analysis

In [228]:
import pandasql as psql
query = """
SELECT batsman ,batsman_id,COUNT(inning) as total_innings, SUM(runs) as total_runs ,SUM(balls_faced) as total_balls
FROM batsman_lvl_data
GROUP BY batsman_id
"""
# Execute the SQL query
result = psql.sqldf(query, locals())

result['strike_rate'] = (result['total_runs'] * 100) / result['total_balls']
result['average_run_per_inning'] = result['total_runs'] / result['total_innings']
bat_csv = result
bat_csv

,batsman,batsman_id,total_innings,total_runs,total_balls,strike_rate,average_run_per_inning
0,DG Be,31464.0,3,10,18,55.555556,3.333333
1,DT Cn,34061.0,54,678,505,134.257426,12.555556
2,CJ Fn,36665.0,10,208,171,121.637427,20.800000
3,AJ Fh,37351.0,62,1590,1298,122.496148,25.645161
4,MC Hs,41740.0,44,1011,788,128.299492,22.977273
...,...,...,...,...,...,...,...
2113,Sv Dy,9822322.0,1,0,1,0.000000,0.000000
2114,PR Vr,9822392.0,1,0,1,0.000000,0.000000
2115,J Be,9822413.0,1,2,4,50.000000,2.000000
2116,BJ Js,9874052.0,1,42,20,210.000000,42.000000


In [229]:
# Calculate percentiles using np.percentile
percentile_40_innings = np.percentile(bat_csv['total_innings'], 40)
percentile_70_strike_rate = np.percentile(bat_csv['strike_rate'], 70)
percentile_60_average_run = np.percentile(bat_csv['average_run_per_inning'], 60)

# Create a new column 'rank' with initial value 0
bat_csv['rank'] = 0

# Set the value of 'rank' to 1 where the conditions are met
bat_csv.loc[(bat_csv['total_innings'] > percentile_40_innings) & 
            (bat_csv['strike_rate'] > percentile_70_strike_rate) & 
            (bat_csv['average_run_per_inning'] > percentile_60_average_run), 'rank'] = 1

# Count the number of rows where 'rank' equals 1
rank_count = bat_csv['rank'].sum()
print(rank_count)
bat_csv

349


,batsman,batsman_id,total_innings,total_runs,total_balls,strike_rate,average_run_per_inning,rank
0,DG Be,31464.0,3,10,18,55.555556,3.333333,0
1,DT Cn,34061.0,54,678,505,134.257426,12.555556,0
2,CJ Fn,36665.0,10,208,171,121.637427,20.800000,0
3,AJ Fh,37351.0,62,1590,1298,122.496148,25.645161,0
4,MC Hs,41740.0,44,1011,788,128.299492,22.977273,1
...,...,...,...,...,...,...,...,...
2113,Sv Dy,9822322.0,1,0,1,0.000000,0.000000,0
2114,PR Vr,9822392.0,1,0,1,0.000000,0.000000,0
2115,J Be,9822413.0,1,2,4,50.000000,2.000000,0
2116,BJ Js,9874052.0,1,42,20,210.000000,42.000000,0


# Bowler_Analysis

In [230]:
query = """
SELECT bowler ,bowler_id,COUNT(inning) as total_innings, SUM(runs) as total_runs ,SUM(balls_bowled) as total_balls, SUM(wicket_count) as total_wicket
FROM bowler_lvl_data
GROUP BY bowler_id
"""
# Execute the SQL query
result = psql.sqldf(query, locals())

result['economy'] = (result['total_runs']) / result['total_balls']*6
result['average'] = (result['total_runs']) / result['total_wicket']
result['strike_rate_bow'] = (result['total_balls']) / result['total_wicket']
bow_csv = result
mean_value = bow_csv['average'][~np.isinf(bow_csv['average'])].mean()
bow_csv['average'] = bow_csv['average'].replace([np.inf, -np.inf], mean_value)
mean_value = bow_csv['strike_rate_bow'][~np.isinf(bow_csv['strike_rate_bow'])].mean()
bow_csv['strike_rate_bow'] = bow_csv['strike_rate_bow'].replace([np.inf, -np.inf], mean_value)
bow_csv

,bowler,bowler_id,total_innings,total_runs,total_balls,total_wicket,economy,average,strike_rate_bow
0,DT Cn,34061.0,49,983,665,29,8.869173,33.896552,22.931034
1,AJ Fh,37351.0,1,10,6,0,10.000000,30.012755,22.069157
2,MC Hs,41740.0,11,92,96,5,5.750000,18.400000,19.200000
3,SJ Oe,49496.0,26,632,582,31,6.515464,20.387097,18.774194
4,PM Se,55299.0,43,1249,873,57,8.584192,21.912281,15.315789
...,...,...,...,...,...,...,...,...,...
1536,PR Vr,9822392.0,1,7,18,3,2.333333,2.333333,6.000000
1537,J Be,9822413.0,1,28,24,0,7.000000,30.012755,22.069157
1538,PS Pa,9822420.0,1,11,6,0,11.000000,30.012755,22.069157
1539,K Ta,9822441.0,2,74,47,2,9.446809,37.000000,23.500000


In [231]:
# Calculate percentiles using np.percentile
percentile_30_innings = np.percentile(bow_csv['total_balls'], 30)
percentile_25_strike_rate = np.percentile(bow_csv['strike_rate_bow'], 40)
percentile_40_economy = np.percentile(bow_csv['economy'], 50)

# Create a new column 'rank' with initial value 0
bow_csv['rank'] = 0

# Set the value of 'rank' to 1 where the conditions are met
bow_csv.loc[(bow_csv['total_balls'] > percentile_30_innings) & 
            (bow_csv['strike_rate_bow'] < percentile_25_strike_rate) & 
            (bow_csv['economy'] < percentile_40_economy), 'rank'] = 1

# Count the number of rows where 'rank' equals 1
rank_count = bow_csv['rank'].sum()
print(rank_count)
bow_csv

249


,bowler,bowler_id,total_innings,total_runs,total_balls,total_wicket,economy,average,strike_rate_bow,rank
0,DT Cn,34061.0,49,983,665,29,8.869173,33.896552,22.931034,0
1,AJ Fh,37351.0,1,10,6,0,10.000000,30.012755,22.069157,0
2,MC Hs,41740.0,11,92,96,5,5.750000,18.400000,19.200000,0
3,SJ Oe,49496.0,26,632,582,31,6.515464,20.387097,18.774194,0
4,PM Se,55299.0,43,1249,873,57,8.584192,21.912281,15.315789,0
...,...,...,...,...,...,...,...,...,...,...
1536,PR Vr,9822392.0,1,7,18,3,2.333333,2.333333,6.000000,0
1537,J Be,9822413.0,1,28,24,0,7.000000,30.012755,22.069157,0
1538,PS Pa,9822420.0,1,11,6,0,11.000000,30.012755,22.069157,0
1539,K Ta,9822441.0,2,74,47,2,9.446809,37.000000,23.500000,0


# Match_data_Analysis

In [232]:
match_lvl_data 

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,2021-01-01,night match,Sr Sh,2020/21,17681,Dn Mw,TJ Pe,148,6,130,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0:3834305.0:3776849.0:6718340.0:3566240.0:4950364.0:7543647.0,18570,2653993.0:6718326.0:6718382.0:2486896.0:2288789.0:4950224.0:8187878.0:3083667.0:4950294.0:6718396.0:1585464.0,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,2021-01-01,night match,Bg Bh Le,2020/21,6663,D Tr,PJ Ge,166,6,106,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1594319.0:7534687.0:7537067.0:1905847.0:8339701.0:7620346.0,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652.0:5788320.0:1612610.0:5509524.0:5497274.0:3876613.0:8193310.0,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,2021-01-02,night match,Bg Bh Le,2020/21,1476,GJ Dn,PJ Ge,165,8,126,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:3063696.0:34061.0:3200756.0:4756982.0:7455818.0:49496.0:1506077.0,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803.0:2535420.0:4967738.0:3995991.0:3890984.0:7353828.0:6818776.0:5419546.0,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,2021-01-02,night match,Sr Sh,2020/21,17681,GA St,TJ Pe,153,7,128,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0:3834305.0:3776849.0:7543647.0:6718340.0:3566240.0:7907458.0,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498.0:6129276.0:2666705.0:6317142.0:3834375.0:2236086.0:4195827.0,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,2021-01-02,day/night match,Bg Bh Le,2020/21,6348,GA Ad,NR Je,164,5,126,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948.0:6249256.0:2340372.0:309056.0:3125849.0:6732004.0:6722540.0,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127.0:2720759.0:2337117.0:6308098.0:7877232.0:3731307.0:4985546.0:2336473.0,other_domestic,33928,3125849.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,9128601,Pb Ks,Ci Sr Ks,Pb Ks,runs,11.0,Ci Sr Ks,field,We Sm Mi,Mumbai,2022-04-25,night match,In Pr Le,2022,5004,M Es,Tn Sa,187,4,133,176.0,6.0,126.0,30407,2789079.0:197658.0:2398346.0:2827327.0:2082044.0:5053082.0:2035102.0:3851518.0:7453697.0:7881845.0:3068547.0,30414,7422673.0:249087.0:3519011.0:5001170.0:232000.0:1642738.0:196580.0:2294823.0:360086.0:7879829.0:7968225.0,In Pr Le,30407,197658.0
1685,9433241,Mx,St,St,wickets,9.0,St,field,Te Cr As Cy Gd Tn,Taunton,2023-06-02,day/night match,Vy Bt,2023,1042,BV Tr,Mk Nl,136,10,115,142.0,2.0,81.0,8700,4421689.0:7752989.0:1941743.0:4489722.0:7672874.0:4005406.0:7671166.0:4489708.0:8820538.0:2593758.0:3590362.0,9701,6139370.0:7694581.0:3294444.0:3239102.0:6320362.0:2535420.0:5288226.0:3252395.0:334963.0:2850868.0:55299.0,other_domestic,9701,NaN
1686,9097227,Bd,Wn Pe,Wn Pe,wickets,6.0,Bd,bat,St Gs Pk Pt Eh,Port Elizabeth,2022-02-07,day match,CA T0 Ce,2021/22,1224,BM We,S Ge,142,6,126,143.0,4.0,105.0,22497,4239773.0:1941743.0:3007969.0:4172972.0:1556253.0:3845029.0:2625895.0:3062555.0:4175772.0:6639156.0:4874092.0,23869,323049.0:4876122.0:4164978.0:1837205.0:3373138.0:2204950.0:1858961.0:6640164.0:2659502.0:4159658.0:4239493.0,other_domestic,23869,323049.0
1687,9516695,Rn Rs,Ss Hd,Ss Hd,wickets,4.0,Rn Rs,bat,Si Mh Sm Jr,Jaipur,2023-05-07,night match,In Pr Le,2023,4661,Nn Mn,Vd Sn,214,2,125,217.0,6.0,125.0,30428,8058959.0:2162782.0:2981614.0:4690188.0:2125696.0:8228429.0:184960.0:3068547.0:3696482.0:8445478.0:3011735.0:6347494.0,36014,5958840

In [233]:
# Get the unique ground IDs
unique_ground_ids = match_lvl_data['ground_id'].unique()

# Initialize an empty list to store win probabilities
win_probabilities = []

# Loop over each unique ground ID
for ground_id in unique_ground_ids:
    # Filter the DataFrame for matches played on the current ground
    matches_on_ground = match_lvl_data[match_lvl_data['ground_id'] == ground_id]
    
    # Count the number of matches where team batting first won
    batting_first_wins = matches_on_ground[(matches_on_ground['by'] == 'runs')] 

    # Calculate the total number of matches played on the ground
    total_matches = len(matches_on_ground)

    # Calculate win probability
    win_probability = len(batting_first_wins) / total_matches 
    
    # Append the win probability to the list
    win_probabilities.append(win_probability)

                                            
win_probabilities_array = np.array(win_probabilities)
# Print the list of win probabilities
win_df = pd.DataFrame({'ground_id': unique_ground_ids, 'win_probability': win_probabilities})
win_df

,ground_id,win_probability
0,17681,0.333333
1,6663,0.333333
2,1476,0.600000
3,6348,0.533333
4,5081,0.545455
...,...,...
146,13453,0.000000
147,7475,1.000000
148,16806,0.000000
149,17632,0.000000


In [235]:
# Get the unique ground IDs
unique_ground_ids = match_lvl_data['team1_id'].unique()

# Initialize an empty list to store win probabilities
win_probabilities = []

# Loop over each unique ground ID
for team1_id in unique_ground_ids:
    # Filter the DataFrame for matches played on the current ground
    matches_on_ground = match_lvl_data[match_lvl_data['team1_id'] == team1_id]
    
    # Count the number of matches where team batting first won
    batting_first_wins = matches_on_ground[(matches_on_ground['by'] == 'runs')] 

    # Calculate the total number of matches played on the ground
    total_matches = len(matches_on_ground)

    # Calculate win probability
    win_probability = len(batting_first_wins) / total_matches 
    
    # Append the win probability to the list
    win_probabilities.append(win_probability)

                                            
win_probabilities_array = np.array(win_probabilities)
# Print the list of win probabilities
win_df_team = pd.DataFrame({'team1_id': unique_ground_ids, 'win_probability11': win_probabilities})
win_df_team

,team1_id,win_probability11
0,17982,0.400000
1,33942,0.461538
2,33956,0.722222
3,33928,0.533333
4,17583,0.500000
...,...,...
162,49650,0.250000
163,132,0.000000
164,237,0.500000
165,1490,0.333333


# Train_data_analysis

In [236]:
# Split the 'team1_roster_ids' column by ':' and expand into separate columns
df = train_data['team1_roster_ids'].str.split(':', expand=True)

# Rename the columns (optional)
df.columns = [f'player_{i+1}' for i in range(df.shape[1])]

train_data = pd.concat([train_data, df], axis=1)

# Drop the 'team1_roster_ids' column from the original train_data DataFrame
train_data = train_data.drop(columns=['team1_roster_ids'])

# Split the 'team1_roster_ids' column by ':' and expand into separate columns
df = train_data['team2_roster_ids'].str.split(':', expand=True)

# Rename the columns (optional)
df.columns = [f'player2_{i+1}' for i in range(df.shape[1])]

train_data = pd.concat([train_data, df], axis=1)

# Drop the 'team1_roster_ids' column from the original train_data DataFrame
train_data = train_data.drop(columns=['team2_roster_ids'])

train_data = train_data.drop(columns=['player_12', 'player2_12'])
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)
# Print the resulting DataFrame
train_data


,match id,team1,team1_id,team2,team2_id,winner,winner_id,winner.1,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11
0,9331181,Ba,11283,Hl Ph,12634,Hl Ph,12634,0,Hl Ph,field,Hr Ct Sm Ie,Indore,20-10-2022,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.00,157.178571,9373356.0,7857520.0,4232164.0,4566540.0,3299407.0,232000.0,7883504.0,8725723.0,7878989.0,2526390.0,8725730.0,3500958.0,4231751.0,2735081.0,2035102.0,3698337.0,5058192.0,6844718.0,4566526.0,6596582.0,4161975.0,7952804.0
1,8797060,Ed,20,Wt Is,41,Ed,20,1,Wt Is,field,Kn Ol Bn Bs,Bridgetown,23-01-2022,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.00,103.500000,2089079.0,6139370.0,2076192.0,62432.0,2083409.0,172199.0,2022957.0,1482998.0,1711492.0,4507208.0,3231437.0,4690258.0,4069666.0,4230127.0,1942317.0,1613926.0,2740408.0,4739552.0,5744850.0,4690104.0,3715697.0,3468870.0
2,9433269,We,10576,Ne,8987,We,10576,1,Ne,field,Tt Be Nm,Nottingham,02-06-2023,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.00,154.333333,3298427.0,2288789.0,7773338.0,3519011.0,3681957.0,7671313.0,5320118.0,2922611.0,7184232.0,7661996.0,6240632.0,4003390.0,1749075.0,1626526.0,4172447.0,5516720.0,7328110.0,125915.0,130437.0,7507303.0,4420828.0,5652828.0
3,9587073,Ga An Ws,36084,Bs Rs,36070,Ga An Ws,36084,1,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,10-09-2023,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.00,144.250000,8127230.0,4690328.0,4069666.0,7960847.0,4690188.0,4739580.0,4739552.0,5744850.0,2294823.0,9266102.0,284339.0,3462080.0,2436405.0,1798705.0,7550857.0,5742470.0,2740408.0,3575774.0,3200756.0,334963.0,6399210.0,6347494.0
4,9516457,Pb Ks,30407,Gt Ts,48341,Gt Ts,48341,0,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,13-04-2023,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.00,189.000000,8127181.0,197658.0,4239038.0,2398346.0,5053082.0,4640824.0,5038046.0,8180500.0,2035102.0,3851518.0,7881845.0,1958683.0,7491224.0,8059029.0,4377610.0,2252452.0,2966879.0,5554254.0,4690230.0,3373285.0,3759846.0,6496482.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,Ci Sr Ks,30414,Pb Ks,30407,1,Ci Sr Ks,field,We Sm Mi,Mumbai,25-04-2022,night match,In Pr Le,2022,5004,0.823529,1.000000,147.333333,66.67,166.400000,2789079.0,197658.0,2398346.0,2827327.0,2082044.0,5053082.0,2035102.0,3851518.0,7453697.0,7881845.0,3068547.0,7422673.0,249087.0,3519011.0,5001170.0,232000.0,1642738.0,196580.0,2294823.0,360086.0,7879829.0,7968225.0
944,9433241,Mx,8700,St,9701,St,9701,0,St,field,Te Cr As Cy Gd Tn,Taunton,02-06-2023,day/night match,Vy Bt,2023,1042,1.571429,0.012346,167.400000,0.00,170.466667,4421689.0,7752989.0,1941743.0,4489722.0,7672874.0,4005406.0,7671166.0,4489708.0,8820538.0,2593758.0,3590362.0,6139370.0,7694581.0,3294444.0,3239102.0,6320362.0,2535420.0,5288226.0,3252395.0,334963.0,2850868.0,55299.0
945,9097227,Bd,22497,Wn Pe,23869,Wn Pe,23869,0,Bd,bat,St Gs Pk Pt Eh,Port Elizabeth,07-02-2022,day match,CA T0 Ce,2021/22,1224,3.000000,1.000000,NaN,0.00,NaN,4239773.0,1941743.0,3007969.0,4172972.0,1556253.0,3845029.0,2625895.0,3062555.0,4175772.0,6639156.0,4874092.0,323049.0,4876122.0,4164978.0,1837205.0,3373138.0,2204950.0,1858961.0,6640164.0,2659502.0,4159658.0,4239493.0
946,9516695,Rn Rs,30428,Ss Hd,36014,Ss Hd,36014,0,Rn Rs,bat,Si Mh Sm Jr,Jaipur,07-05-2023,night match,In Pr Le,2023,4661,0.789474,1.487805,182.800000,66.67,133.375000,8058959.0,2162782.0,2981614.0,4690188.0,2125696.0,8228429.0,184960.0,3068547.0,3696482

# Creating_star_players_column_in_train_data

In [237]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(train_data)):
    count1 = 0
    count2 = 0
    
    for col in column_names_team1:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count1 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count1 += corresponding_value
    
    for col in column_names_team2:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count2 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count2 += corresponding_value
    
    # Calculate the difference and assign to 'star_players' column
    train_data.at[j, 'star_players'] = count1 - count2

# Count the number of rows where 'star_players' > 2 or 'star_players' < -2
count = ((train_data['star_players'] > 2) | (train_data['star_players'] < -2)).sum()

# Print the count
print("Number of rows where 'star_players' > 2 or 'star_players' < -2:", count)

Number of rows where 'star_players' > 2 or 'star_players' < -2: 305


# Creating_average_average(Batsman) in train_data

In [238]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(train_data)):
    count1 = 0
    count2 = 0
    number1=0
    number2=0 
    for col in column_names_team1:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty and (filtered_df['total_balls'].gt(10).any()):
            corresponding_value = filtered_df['average_run_per_inning'].iloc[0]
            count1 += corresponding_value
            number1 += 1
    
    for col in column_names_team2:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty and (filtered_df['total_balls'].gt(10).any()):
            corresponding_value = filtered_df['average_run_per_inning'].iloc[0]
            count2 += corresponding_value
            number2 += 1
    
    # Calculate the difference and assign to 'star_players' column
    train_data.at[j, 'average_average'] = (count1/number1) - (count2/number2)


# Creating_average_strike_rate (Batsman) in train_data

In [239]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(train_data)):
    count1 = 0
    count2 = 0
    number1=0
    number2=0 
    for col in column_names_team1:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            if filtered_df['total_balls'].iloc[0]> 10:
                corresponding_value = filtered_df['strike_rate'].iloc[0]
                count1 += corresponding_value
                number1 += 1
    
    for col in column_names_team2:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            if filtered_df['total_balls'].iloc[0]> 10:
                corresponding_value = filtered_df['strike_rate'].iloc[0]
                count2 += corresponding_value
                number2 += 1
    
    # Calculate the difference and assign to 'star_players' column
    train_data.at[j, 'average_strike_rate'] = (count1/number1) - ( count2/number2)



# Creating_average_economy (Bowler) in train_data

In [240]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Ensure 'average_economy' column exists in train_data
train_data['average_economy'] = None

# Iterate through the rows of the DataFrame
for j in range(len(train_data)):
    count1 = 0
    count2 = 0
    number1 = 0
    number2 = 0
    
    for col in column_names_team1:
        x = train_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            if filtered_df['total_balls'].iloc[0]> 10:
                corresponding_value = filtered_df['economy'].iloc[0]
                count1 += corresponding_value
                number1 += 1
    
    for col in column_names_team2:
        x = train_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            if filtered_df['total_balls'].iloc[0]> 10:
                corresponding_value = filtered_df['economy'].iloc[0]
                count2 += corresponding_value
                number2 += 1
    
    if number1 == 0:
        average_economy_team1 = 1
    else:
        average_economy_team1 = count1 / number1
    
    if number2 == 0:
        average_economy_team2 = -1
    else:
        average_economy_team2 = count2 / number2
    
    # Calculate the difference and assign to 'average_economy' column
    train_data.at[j, 'average_economy'] = average_economy_team1 - average_economy_team2


# Creating_average_bowler_average (Bowler) in train_data

In [241]:
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Ensure 'average_economy' column exists in train_data
train_data['average_bowler_average'] = None

# Iterate through the rows of the DataFrame
for j in range(len(train_data)):
    count1 = 0
    count2 = 0
    number1 = 0
    number2 = 0
    
    for col in column_names_team1:
        x = train_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            if filtered_df['total_balls'].iloc[0]> 10:
                corresponding_value = filtered_df['average'].iloc[0]
                count1 += corresponding_value
                number1 += 1
    
    for col in column_names_team2:
        x = train_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            if filtered_df['total_balls'].iloc[0]> 10:
                corresponding_value = filtered_df['average'].iloc[0]
                count2 += corresponding_value
                number2 += 1
    
    if number1 == 0:
        average_bowler_average_team1 = 10
    else:
        average_bowler_average_team1 = count1 / number1
    
    if number2 == 0:
        average_bowler_average_team2 = -10
    else:
        average_bowler_average_team2 = count2 / number2
    
    # Calculate the difference and assign to 'average_economy' column
    train_data.at[j, 'average_bowler_average'] = average_bowler_average_team1 - average_bowler_average_team2


# Creating total_innings (players) in train_data

In [242]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(train_data)):
    count1 = 0
    count2 = 0
    
    for col in column_names_team1:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count1 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count1 += corresponding_value
    
    for col in column_names_team2:
        x = train_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count2 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count2 += corresponding_value
    
    # Calculate the difference and assign to 'star_players' column
    train_data.at[j, 'total_innings_diff'] = count1 - count2
    
    train_data

In [243]:
train_data['toss_win'] = train_data.apply(lambda row: 1 if row['team1'] == row['toss winner'] else 0, axis=1)
train_data

,match id,team1,team1_id,team2,team2_id,winner,winner_id,winner.1,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11,star_players,average_average,average_strike_rate,average_economy,average_bowler_average,total_innings_diff,toss_win
0,9331181,Ba,11283,Hl Ph,12634,Hl Ph,12634,0,Hl Ph,field,Hr Ct Sm Ie,Indore,20-10-2022,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.00,157.178571,9373356.0,7857520.0,4232164.0,4566540.0,3299407.0,232000.0,7883504.0,8725723.0,7878989.0,2526390.0,8725730.0,3500958.0,4231751.0,2735081.0,2035102.0,3698337.0,5058192.0,6844718.0,4566526.0,6596582.0,4161975.0,7952804.0,1.0,-7.163878,-1.512755,-0.450542,27.716635,129.0,0
1,8797060,Ed,20,Wt Is,41,Ed,20,1,Wt Is,field,Kn Ol Bn Bs,Bridgetown,23-01-2022,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.00,103.500000,2089079.0,6139370.0,2076192.0,62432.0,2083409.0,172199.0,2022957.0,1482998.0,1711492.0,4507208.0,3231437.0,4690258.0,4069666.0,4230127.0,1942317.0,1613926.0,2740408.0,4739552.0,5744850.0,4690104.0,3715697.0,3468870.0,5.0,0.382930,-4.331673,-0.511835,-3.045311,147.0,0
2,9433269,We,10576,Ne,8987,We,10576,1,Ne,field,Tt Be Nm,Nottingham,02-06-2023,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.00,154.333333,3298427.0,2288789.0,7773338.0,3519011.0,3681957.0,7671313.0,5320118.0,2922611.0,7184232.0,7661996.0,6240632.0,4003390.0,1749075.0,1626526.0,4172447.0,5516720.0,7328110.0,125915.0,130437.0,7507303.0,4420828.0,5652828.0,1.0,0.002155,-11.592568,-0.119164,-2.464579,-256.0,0
3,9587073,Ga An Ws,36084,Bs Rs,36070,Ga An Ws,36084,1,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,10-09-2023,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.00,144.250000,8127230.0,4690328.0,4069666.0,7960847.0,4690188.0,4739580.0,4739552.0,5744850.0,2294823.0,9266102.0,284339.0,3462080.0,2436405.0,1798705.0,7550857.0,5742470.0,2740408.0,3575774.0,3200756.0,334963.0,6399210.0,6347494.0,1.0,1.692551,6.717759,0.055254,-1.999591,-176.0,1
4,9516457,Pb Ks,30407,Gt Ts,48341,Gt Ts,48341,0,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,13-04-2023,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.00,189.000000,8127181.0,197658.0,4239038.0,2398346.0,5053082.0,4640824.0,5038046.0,8180500.0,2035102.0,3851518.0,7881845.0,1958683.0,7491224.0,8059029.0,4377610.0,2252452.0,2966879.0,5554254.0,4690230.0,3373285.0,3759846.0,6496482.0,-2.0,-1.690841,9.710462,0.117038,-1.510439,-115.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,Ci Sr Ks,30414,Pb Ks,30407,1,Ci Sr Ks,field,We Sm Mi,Mumbai,25-04-2022,night match,In Pr Le,2022,5004,0.823529,1.000000,147.333333,66.67,166.400000,2789079.0,197658.0,2398346.0,2827327.0,2082044.0,5053082.0,2035102.0,3851518.0,7453697.0,7881845.0,3068547.0,7422673.0,249087.0,3519011.0,5001170.0,232000.0,1642738.0,196580.0,2294823.0,360086.0,7879829.0,7968225.0,0.0,0.737198,1.323546,0.096289,0.92542,-21.0,0
944,9433241,Mx,8700,St,9701,St,9701,0,St,field,Te Cr As Cy Gd Tn,Taunton,02-06-2023,day/night match,Vy Bt,2023,1042,1.571429,0.012346,167.400000,0.00,170.466667,4421689.0,7752989.0,1941743.0,4489722.0,7672874.0,4005406.0,7671166.0,4489708.0,8820538.0,2593758.0,3590362.0,6139370.0,7694581.0,3294444.0,3239102.0,6320362.0,2535420.0,5288226.0,3252395.0,334963.0,2850868.0,55299.0,-1.0,2.572686,-7.787072,1.71305,5.000097,-422.0,0
945,9097227,Bd,22497,Wn Pe,23869,Wn Pe,23869,0,Bd,bat,St Gs Pk Pt Eh,Port Elizabeth,07-02-2022,day match,CA T0 Ce,2021/22,1224,3.00

# Ground_Id

In [244]:
# Merge train_data with win_df based on 'ground_id'
merged_data = pd.merge(train_data, win_df, on='ground_id', how='left')

# Fill missing win probabilities with a default value (e.g., 0) if any
merged_data['win_probability'].fillna(0, inplace=True)

# Print the merged DataFrame to verify the changes
train_data = merged_data
train_data

,match id,team1,team1_id,team2,team2_id,winner,winner_id,winner.1,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11,star_players,average_average,average_strike_rate,average_economy,average_bowler_average,total_innings_diff,toss_win,win_probability
0,9331181,Ba,11283,Hl Ph,12634,Hl Ph,12634,0,Hl Ph,field,Hr Ct Sm Ie,Indore,20-10-2022,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.00,157.178571,9373356.0,7857520.0,4232164.0,4566540.0,3299407.0,232000.0,7883504.0,8725723.0,7878989.0,2526390.0,8725730.0,3500958.0,4231751.0,2735081.0,2035102.0,3698337.0,5058192.0,6844718.0,4566526.0,6596582.0,4161975.0,7952804.0,1.0,-7.163878,-1.512755,-0.450542,27.716635,129.0,0,0.500000
1,8797060,Ed,20,Wt Is,41,Ed,20,1,Wt Is,field,Kn Ol Bn Bs,Bridgetown,23-01-2022,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.00,103.500000,2089079.0,6139370.0,2076192.0,62432.0,2083409.0,172199.0,2022957.0,1482998.0,1711492.0,4507208.0,3231437.0,4690258.0,4069666.0,4230127.0,1942317.0,1613926.0,2740408.0,4739552.0,5744850.0,4690104.0,3715697.0,3468870.0,5.0,0.382930,-4.331673,-0.511835,-3.045311,147.0,0,0.636364
2,9433269,We,10576,Ne,8987,We,10576,1,Ne,field,Tt Be Nm,Nottingham,02-06-2023,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.00,154.333333,3298427.0,2288789.0,7773338.0,3519011.0,3681957.0,7671313.0,5320118.0,2922611.0,7184232.0,7661996.0,6240632.0,4003390.0,1749075.0,1626526.0,4172447.0,5516720.0,7328110.0,125915.0,130437.0,7507303.0,4420828.0,5652828.0,1.0,0.002155,-11.592568,-0.119164,-2.464579,-256.0,0,0.586207
3,9587073,Ga An Ws,36084,Bs Rs,36070,Ga An Ws,36084,1,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,10-09-2023,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.00,144.250000,8127230.0,4690328.0,4069666.0,7960847.0,4690188.0,4739580.0,4739552.0,5744850.0,2294823.0,9266102.0,284339.0,3462080.0,2436405.0,1798705.0,7550857.0,5742470.0,2740408.0,3575774.0,3200756.0,334963.0,6399210.0,6347494.0,1.0,1.692551,6.717759,0.055254,-1.999591,-176.0,1,0.555556
4,9516457,Pb Ks,30407,Gt Ts,48341,Gt Ts,48341,0,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,13-04-2023,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.00,189.000000,8127181.0,197658.0,4239038.0,2398346.0,5053082.0,4640824.0,5038046.0,8180500.0,2035102.0,3851518.0,7881845.0,1958683.0,7491224.0,8059029.0,4377610.0,2252452.0,2966879.0,5554254.0,4690230.0,3373285.0,3759846.0,6496482.0,-2.0,-1.690841,9.710462,0.117038,-1.510439,-115.0,0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,Ci Sr Ks,30414,Pb Ks,30407,1,Ci Sr Ks,field,We Sm Mi,Mumbai,25-04-2022,night match,In Pr Le,2022,5004,0.823529,1.000000,147.333333,66.67,166.400000,2789079.0,197658.0,2398346.0,2827327.0,2082044.0,5053082.0,2035102.0,3851518.0,7453697.0,7881845.0,3068547.0,7422673.0,249087.0,3519011.0,5001170.0,232000.0,1642738.0,196580.0,2294823.0,360086.0,7879829.0,7968225.0,0.0,0.737198,1.323546,0.096289,0.92542,-21.0,0,0.358974
944,9433241,Mx,8700,St,9701,St,9701,0,St,field,Te Cr As Cy Gd Tn,Taunton,02-06-2023,day/night match,Vy Bt,2023,1042,1.571429,0.012346,167.400000,0.00,170.466667,4421689.0,7752989.0,1941743.0,4489722.0,7672874.0,4005406.0,7671166.0,4489708.0,8820538.0,2593758.0,3590362.0,6139370.0,7694581.0,3294444.0,3239102.0,6320362.0,2535420.0,5288226.0,3252395.0,334963.0,2850868.0,55299.0,-1.0,2.572686,-7.787072,1.71305,5.000097,-422.0,0,0.631579
945,9097227,Bd,22497,Wn Pe,23869,Wn Pe,23869,0,B

In [245]:
# Merge train_data with win_df based on 'ground_id'
merged_data = pd.merge(train_data, win_df_team, on='team1_id', how='left')

# Fill missing win probabilities with a default value (e.g., 0) if any
merged_data['win_probability11'].fillna(0, inplace=True)

# Print the merged DataFrame to verify the changes
train_data = merged_data
train_data

,match id,team1,team1_id,team2,team2_id,winner,winner_id,winner.1,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11,star_players,average_average,average_strike_rate,average_economy,average_bowler_average,total_innings_diff,toss_win,win_probability,win_probability11
0,9331181,Ba,11283,Hl Ph,12634,Hl Ph,12634,0,Hl Ph,field,Hr Ct Sm Ie,Indore,20-10-2022,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.00,157.178571,9373356.0,7857520.0,4232164.0,4566540.0,3299407.0,232000.0,7883504.0,8725723.0,7878989.0,2526390.0,8725730.0,3500958.0,4231751.0,2735081.0,2035102.0,3698337.0,5058192.0,6844718.0,4566526.0,6596582.0,4161975.0,7952804.0,1.0,-7.163878,-1.512755,-0.450542,27.716635,129.0,0,0.500000,0.500000
1,8797060,Ed,20,Wt Is,41,Ed,20,1,Wt Is,field,Kn Ol Bn Bs,Bridgetown,23-01-2022,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.00,103.500000,2089079.0,6139370.0,2076192.0,62432.0,2083409.0,172199.0,2022957.0,1482998.0,1711492.0,4507208.0,3231437.0,4690258.0,4069666.0,4230127.0,1942317.0,1613926.0,2740408.0,4739552.0,5744850.0,4690104.0,3715697.0,3468870.0,5.0,0.382930,-4.331673,-0.511835,-3.045311,147.0,0,0.636364,0.619048
2,9433269,We,10576,Ne,8987,We,10576,1,Ne,field,Tt Be Nm,Nottingham,02-06-2023,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.00,154.333333,3298427.0,2288789.0,7773338.0,3519011.0,3681957.0,7671313.0,5320118.0,2922611.0,7184232.0,7661996.0,6240632.0,4003390.0,1749075.0,1626526.0,4172447.0,5516720.0,7328110.0,125915.0,130437.0,7507303.0,4420828.0,5652828.0,1.0,0.002155,-11.592568,-0.119164,-2.464579,-256.0,0,0.586207,0.461538
3,9587073,Ga An Ws,36084,Bs Rs,36070,Ga An Ws,36084,1,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,10-09-2023,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.00,144.250000,8127230.0,4690328.0,4069666.0,7960847.0,4690188.0,4739580.0,4739552.0,5744850.0,2294823.0,9266102.0,284339.0,3462080.0,2436405.0,1798705.0,7550857.0,5742470.0,2740408.0,3575774.0,3200756.0,334963.0,6399210.0,6347494.0,1.0,1.692551,6.717759,0.055254,-1.999591,-176.0,1,0.555556,0.642857
4,9516457,Pb Ks,30407,Gt Ts,48341,Gt Ts,48341,0,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,13-04-2023,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.00,189.000000,8127181.0,197658.0,4239038.0,2398346.0,5053082.0,4640824.0,5038046.0,8180500.0,2035102.0,3851518.0,7881845.0,1958683.0,7491224.0,8059029.0,4377610.0,2252452.0,2966879.0,5554254.0,4690230.0,3373285.0,3759846.0,6496482.0,-2.0,-1.690841,9.710462,0.117038,-1.510439,-115.0,0,0.333333,0.409091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,Ci Sr Ks,30414,Pb Ks,30407,1,Ci Sr Ks,field,We Sm Mi,Mumbai,25-04-2022,night match,In Pr Le,2022,5004,0.823529,1.000000,147.333333,66.67,166.400000,2789079.0,197658.0,2398346.0,2827327.0,2082044.0,5053082.0,2035102.0,3851518.0,7453697.0,7881845.0,3068547.0,7422673.0,249087.0,3519011.0,5001170.0,232000.0,1642738.0,196580.0,2294823.0,360086.0,7879829.0,7968225.0,0.0,0.737198,1.323546,0.096289,0.92542,-21.0,0,0.358974,0.409091
944,9433241,Mx,8700,St,9701,St,9701,0,St,field,Te Cr As Cy Gd Tn,Taunton,02-06-2023,day/night match,Vy Bt,2023,1042,1.571429,0.012346,167.400000,0.00,170.466667,4421689.0,7752989.0,1941743.0,4489722.0,7672874.0,4005406.0,7671166.0,4489708.0,8820538.0,2593758.0,3590362.0,6139370.0,7694581.0,3294444.0,3239102.0,6320362.0,2535420.0,5288226.0,3252395.0,334963.0,2850868.0,55299.0,-1.0,2.572686,-7.787072,1.71305

In [247]:
train_data['team1only_avg_runs_last15']=train_data['team1only_avg_runs_last15']/100

In [248]:
train_data['ground_avg_runs_last15']=train_data['ground_avg_runs_last15']/100
train_data

,match id,team1,team1_id,team2,team2_id,winner,winner_id,winner.1,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11,star_players,average_average,average_strike_rate,average_economy,average_bowler_average,total_innings_diff,toss_win,win_probability,win_probability11,win_probability2
0,9331181,Ba,11283,Hl Ph,12634,Hl Ph,12634,0,Hl Ph,field,Hr Ct Sm Ie,Indore,20-10-2022,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,1.390000,100.00,1.571786,9373356.0,7857520.0,4232164.0,4566540.0,3299407.0,232000.0,7883504.0,8725723.0,7878989.0,2526390.0,8725730.0,3500958.0,4231751.0,2735081.0,2035102.0,3698337.0,5058192.0,6844718.0,4566526.0,6596582.0,4161975.0,7952804.0,1.0,-7.163878,-1.512755,-0.450542,27.716635,129.0,0,0.500000,0.500000,1.000000
1,8797060,Ed,20,Wt Is,41,Ed,20,1,Wt Is,field,Kn Ol Bn Bs,Bridgetown,23-01-2022,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,1.560000,50.00,1.035000,2089079.0,6139370.0,2076192.0,62432.0,2083409.0,172199.0,2022957.0,1482998.0,1711492.0,4507208.0,3231437.0,4690258.0,4069666.0,4230127.0,1942317.0,1613926.0,2740408.0,4739552.0,5744850.0,4690104.0,3715697.0,3468870.0,5.0,0.382930,-4.331673,-0.511835,-3.045311,147.0,0,0.636364,0.619048,0.458333
2,9433269,We,10576,Ne,8987,We,10576,1,Ne,field,Tt Be Nm,Nottingham,02-06-2023,day/night match,Vy Bt,2023,251,0.857143,0.672131,1.732667,0.00,1.543333,3298427.0,2288789.0,7773338.0,3519011.0,3681957.0,7671313.0,5320118.0,2922611.0,7184232.0,7661996.0,6240632.0,4003390.0,1749075.0,1626526.0,4172447.0,5516720.0,7328110.0,125915.0,130437.0,7507303.0,4420828.0,5652828.0,1.0,0.002155,-11.592568,-0.119164,-2.464579,-256.0,0,0.586207,0.461538,0.470588
3,9587073,Ga An Ws,36084,Bs Rs,36070,Ga An Ws,36084,1,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,10-09-2023,day match,Cn Pr Le,2023,14300,2.166667,1.975610,1.642667,50.00,1.442500,8127230.0,4690328.0,4069666.0,7960847.0,4690188.0,4739580.0,4739552.0,5744850.0,2294823.0,9266102.0,284339.0,3462080.0,2436405.0,1798705.0,7550857.0,5742470.0,2740408.0,3575774.0,3200756.0,334963.0,6399210.0,6347494.0,1.0,1.692551,6.717759,0.055254,-1.999591,-176.0,1,0.555556,0.642857,0.461538
4,9516457,Pb Ks,30407,Gt Ts,48341,Gt Ts,48341,0,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,13-04-2023,night match,In Pr Le,2023,7118,0.818182,1.327869,1.646667,0.00,1.890000,8127181.0,197658.0,4239038.0,2398346.0,5053082.0,4640824.0,5038046.0,8180500.0,2035102.0,3851518.0,7881845.0,1958683.0,7491224.0,8059029.0,4377610.0,2252452.0,2966879.0,5554254.0,4690230.0,3373285.0,3759846.0,6496482.0,-2.0,-1.690841,9.710462,0.117038,-1.510439,-115.0,0,0.333333,0.409091,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,Ci Sr Ks,30414,Pb Ks,30407,1,Ci Sr Ks,field,We Sm Mi,Mumbai,25-04-2022,night match,In Pr Le,2022,5004,0.823529,1.000000,1.473333,66.67,1.664000,2789079.0,197658.0,2398346.0,2827327.0,2082044.0,5053082.0,2035102.0,3851518.0,7453697.0,7881845.0,3068547.0,7422673.0,249087.0,3519011.0,5001170.0,232000.0,1642738.0,196580.0,2294823.0,360086.0,7879829.0,7968225.0,0.0,0.737198,1.323546,0.096289,0.92542,-21.0,0,0.358974,0.409091,0.692308
944,9433241,Mx,8700,St,9701,St,9701,0,St,field,Te Cr As Cy Gd Tn,Taunton,02-06-2023,day/night match,Vy Bt,2023,1042,1.571429,0.012346,1.674000,0.00,1.704667,4421689.0,7752989.0,1941743.0,4489722.0,7672874.0,4005406.0,7671166.0,4489708.0,8820538.0,2593758.0,3590362.0,6139370.0,7694581.0,3294444.0,3239102.0,6320362.0,2535420.0,5288226.0,3252395.0,334963.0,28

# Selection of particular columns for training using Xgbooster 

In [278]:
# List of columns to keep
columns_to_keep = ['winner.1', 'team_count_50runs_last15','team_winp_last5','team1only_avg_runs_last15','team1_winp_team2_last15','star_players','average_average','average_economy','average_bowler_average','average_strike_rate','total_innings_diff','toss_win','win_probability','win_probability11']
# Create a new DataFrame with only the specified columns
new_df = train_data[columns_to_keep]


new_df['average_economy'] = pd.to_numeric(new_df['average_economy'], errors='coerce')
new_df['average_bowler_average'] = pd.to_numeric(new_df['average_bowler_average'], errors='coerce')
new_df['average_economy'].fillna(0, inplace=True)
new_df['average_bowler_average'].fillna(0, inplace=True)

new_df


C:\Users\Chitwan\AppData\Local\Temp\ipykernel_3200\3027784794.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['average_economy'] = pd.to_numeric(new_df['average_economy'], errors='coerce')
C:\Users\Chitwan\AppData\Local\Temp\ipykernel_3200\3027784794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['average_bowler_average'] = pd.to_numeric(new_df['average_bowler_average'], errors='coerce')
C:\Users\Chitwan\AppData\Local\Temp\ipykernel_3200\3027784794.py:9: SettingWithCopyWarning: 
A val

,winner.1,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,star_players,average_average,average_economy,average_bowler_average,average_strike_rate,total_innings_diff,toss_win,win_probability,win_probability11
0,0,1.666667,0.672131,1.390000,100.00,1.0,-7.163878,-0.450542,27.716635,-1.512755,129.0,0,0.500000,0.500000
1,1,1.285714,1.952381,1.560000,50.00,5.0,0.382930,-0.511835,-3.045311,-4.331673,147.0,0,0.636364,0.619048
2,1,0.857143,0.672131,1.732667,0.00,1.0,0.002155,-0.119164,-2.464579,-11.592568,-256.0,0,0.586207,0.461538
3,1,2.166667,1.975610,1.642667,50.00,1.0,1.692551,0.055254,-1.999591,6.717759,-176.0,1,0.555556,0.642857
4,0,0.818182,1.327869,1.646667,0.00,-2.0,-1.690841,0.117038,-1.510439,9.710462,-115.0,0,0.333333,0.409091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,1,0.823529,1.000000,1.473333,66.67,0.0,0.737198,0.096289,0.925420,1.323546,-21.0,0,0.358974,0.409091
944,0,1.571429,0.012346,1.674000,0.00,-1.0,2.572686,1.713050,5.000097,-7.787072,-422.0,0,0.631579,0.300000
945,0,3.000000,1.000000,NaN,0.00,-1.0,5.340619,0.041308,-2.539882,15.173058,-160.0,1,0.416667,0.666667
946,0,0.789474,1.487805,1.828000,66.67,-2.0,-0.762494,0.089891,2.702330,-14.120582,-128.0,1,0.411765,0.473684


# Training of new_df using Xgbooster

In [279]:
# from sklearn.model_selection import train_test_split
# import xgboost as xgb
# from sklearn.metrics import accuracy_score
# X = new_df.drop('winner.1', axis=1)
# y = new_df['winner.1']

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [280]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

# Split the data
X = new_df.drop('winner.1', axis=1)
y = new_df['winner.1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Pools for CatBoost
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Set parameters for CatBoost
params = {
    'iterations':100,
    'depth': 3,
    'learning_rate': 0.001,
    'loss_function': 'Logloss',  # Binary classification
    'eval_metric': 'Accuracy'
}

# Train the model
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=test_pool, verbose=False)

# Predict
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred_prob]

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.7105


In [281]:
y_pred_binary = np.array(y_pred_binary)
y_pred_binary.shape

(190,)

In [282]:
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 71.05%


# Test_data analysis

In [254]:
# Split the 'team1_roster_ids' column by ':' and expand into separate columns
df = test_data['team1_roster_ids'].str.split(':', expand=True)

# Rename the columns (optional)
df.columns = [f'player_{i+1}' for i in range(df.shape[1])]

test_data = pd.concat([test_data, df], axis=1)

# Drop the 'team1_roster_ids' column from the original test_data DataFrame
test_data = test_data.drop(columns=['team1_roster_ids'])

# Split the 'team2_roster_ids' column by ':' and expand into separate columns
df = test_data['team2_roster_ids'].str.split(':', expand=True)

# Rename the columns (optional)
df.columns = [f'player2_{i+1}' for i in range(df.shape[1])]

test_data = pd.concat([test_data, df], axis=1)

# Drop the 'team2_roster_ids' column from the original test_data DataFrame
test_data = test_data.drop(columns=['team2_roster_ids'])

test_data = test_data.drop(columns=['player_12', 'player2_12','player2_13'])
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', 0)
# Print the resulting DataFrame
test_data


,match id,team1,team1_id,team2,team2_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11
0,9250275,Jy,28594,Ud Ss of Aa,90,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN,7438955.0,8271969.0,8369661.0,3685247.0,2590251.0,7785497.0,7785553.0,3296264.0,4613622.0,2904796.0,3685289.0,2436944.0,8117500.0,6703528.0,3010748.0,1613870.0,8934764.0,2721480.0,1632819.0,2789604.0,6489846.0,1565507.0
1,9262189,Ga An Ws,36084,Ja Ts,36098,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.500000,3715690.0,6818622.0,4069666.0,393014.0,4690188.0,4739552.0,4690328.0,5744850.0,4739580.0,9266102.0,284339.0,4690258.0,3761246.0,8464385.0,5742470.0,3150720.0,4690104.0,181404.0,1594319.0,5509524.0,4239794.0,2036649.0
2,9128776,Rn Rs,30428,Lw Sr Gs,48334,Rn Rs,bat,Be Sm Mi,Mumbai,2022-05-15,night match,In Pr Le,2022,2764,0.842105,0.753086,171.066667,100.00,179.625000,8058959.0,2162782.0,2981614.0,7833195.0,7556051.0,2486896.0,184960.0,1945397.0,6420126.0,3011735.0,6347494.0,2654014.0,2954769.0,8058903.0,3479860.0,3299407.0,2275097.0,2740408.0,3865077.0,7924048.0,4859490.0,8228100.0
3,9586919,Ja Ts,36098,St La Ks,36112,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,Gros Islet,2023-08-16,night match,Cn Pr Le,2023,11892,0.285714,1.487805,166.000000,75.00,155.500000,4690258.0,8464385.0,1613898.0,5744780.0,3150720.0,4690104.0,1594319.0,5509524.0,2036649.0,3023383.0,5471948.0,2331475.0,313809.0,391103.0,2742837.0,2097017.0,7921535.0,7459647.0,8464343.0,4690230.0,8339701.0,4702774.0
4,9128538,Ci Sr Ks,30414,Gt Ts,48341,Gt Ts,field,Ma Ct An Sm Pe,Pune,2022-04-17,night match,In Pr Le,2022,18752,2.375000,0.310330,169.933333,0.00,164.125000,7422673.0,249087.0,62432.0,232000.0,5001170.0,1642738.0,196580.0,360086.0,2022957.0,7968225.0,7879829.0,1958683.0,7491224.0,3339160.0,5452754.0,2252452.0,2966879.0,5554254.0,4690230.0,3456424.0,3373285.0,8118053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,9094371,Ln St,46773,Wh Fe,46752,Wh Fe,field,Ls Ln,London,2022-08-24,night match,Te Hd Ms Cn,2022,83,0.916667,2.904762,144.545455,0.00,145.181818,3200973.0,2958164.0,4223883.0,172199.0,4489974.0,1613926.0,5367676.0,1482998.0,5788418.0,2173688.0,4626236.0,8364726.0,4003390.0,2158869.0,3651472.0,2252452.0,2294823.0,4170186.0,5436878.0,3913475.0,2538983.0,2904943.0
267,9440500,Si La,69,Ia,55,Ia,field,Ma Ct An Sm Pe,Pune,2023-01-05,night match,Si La tr of Ia,2022/23,18752,0.714286,1.000000,159.066667,40.00,154.950000,7200598.0,4403531.0,2398346.0,5490582.0,3260564.0,3061225.0,5490666.0,4365878.0,7968225.0,3497171.0,5551062.0,5043310.0,7491224.0,3127354.0,3125562.0,4377610.0,3479860.0,3882850.0,7558949.0,8725709.0,3011735.0,7881845.0
268,9085173,Sx,9967,Gn,7573,Gn,field,Sa Gs Cf,Cardiff,2022-06-19,day match,Vy Bt,2022,4521,1.888889,2.904762,174.400000,50.00,146.821429,2263736.0,164233.0,8820496.0,4215098.0,74087.0,4411231.0,6311612.0,5554254.0,3214812.0,8006676.0,1471994.0,2531913.0,1722048.0,319948.0,5164844.0,4174610.0,6196280.0,8035894.0,3998070.0,2633196.0,3212278.0,7664957.0
269,8887752,Sy Tr,33963,Me Rs,33942,Me Rs,field,Ds Sm Me,Melbourne,2022-01-08,night match,Bg Bh Le,2021/22,8252,0.333333,1.487805,171.066667,100.00,161.750000,7572123.0,1749075.0,6718802.0,5788320.0,7534652.0,1612610.0,7886864.0,5497274.0,3497633.0,8193310.0,8106713.0,5406540.0,37351.0,46794.0,2336473.0,7534687.0,5469092.0,7537067.0,1905847.0,1906708.0,7620297.0,4985546.0


# Creating star player column in test_data 

In [255]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(test_data)):
    count1 = 0
    count2 = 0
    
    for col in column_names_team1:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count1 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count1 += corresponding_value
    
    for col in column_names_team2:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count2 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['rank'].iloc[0]
            count2 += corresponding_value
    
    # Calculate the difference and assign to 'star_players' column
    test_data.at[j, 'star_players'] = count1 - count2


# Creating average_average column in test_data

In [256]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(test_data)):
    count1 = 0
    count2 = 0
    number1 = 0
    number2 = 0
    
    for col in column_names_team1:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['average_run_per_inning'].iloc[0]
            count1 += corresponding_value
            number1 += 1
    
    for col in column_names_team2:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['average_run_per_inning'].iloc[0]
            count2 += corresponding_value
            number2 += 1
    
    # Calculate the difference and assign to 'average_average' column
    if number1 != 0 and number2 != 0:  # Check for division by zero
        test_data.at[j, 'average_average'] = (count1 / number1) - (count2 / number2)
    else:
        test_data.at[j, 'average_average'] = float('NaN')


# Creating strike_rate column in test_data

In [257]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(test_data)):
    count1 = 0
    count2 = 0
    number1 = 0
    number2 = 0
    
    for col in column_names_team1:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty and filtered_df['total_balls'].iloc[0] > 10:
            corresponding_value = filtered_df['strike_rate'].iloc[0]
            count1 += corresponding_value
            number1 += 1
    
    for col in column_names_team2:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty and filtered_df['total_balls'].iloc[0] > 10:
            corresponding_value = filtered_df['strike_rate'].iloc[0]
            count2 += corresponding_value
            number2 += 1
    
    # Calculate the difference and assign to 'average_strike_rate' column
    if number1 != 0 and number2 != 0:  # Check for division by zero
        test_data.at[j, 'average_strike_rate'] = (count1 / number1) - (count2 / number2)
    else:
        test_data.at[j, 'average_strike_rate'] = float('NaN')


# Creating average_economy in test_data

In [258]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Ensure 'average_economy' column exists in train_data
test_data['average_economy'] = None

# Iterate through the rows of the DataFrame
for j in range(len(test_data)):
    count1 = 0
    count2 = 0
    number1 = 0
    number2 = 0
    
    for col in column_names_team1:
        x = test_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['economy'].iloc[0]
            count1 += corresponding_value
            number1 += 1
    
    for col in column_names_team2:
        x = test_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['economy'].iloc[0]
            count2 += corresponding_value
            number2 += 1
    
    if number1 == 0:
        average_economy_team1 = 1
    else:
        average_economy_team1 = count1 / number1
    
    if number2 == 0:
        average_economy_team2 = -1
    else:
        average_economy_team2 = count2 / number2
    
    # Calculate the difference and assign to 'average_economy' column
    test_data.at[j, 'average_economy'] = average_economy_team1 - average_economy_team2


# Creating average_bowler_average column in test_data

In [259]:
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Ensure 'average_economy' column exists in train_data
test_data['average_bowler_average'] = None

# Iterate through the rows of the DataFrame
for j in range(len(test_data)):
    count1 = 0
    count2 = 0
    number1 = 0
    number2 = 0
    
    for col in column_names_team1:
        x = test_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['average'].iloc[0]
            count1 += corresponding_value
            number1 += 1
    
    for col in column_names_team2:
        x = test_data[col].iloc[j]
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['average'].iloc[0]
            count2 += corresponding_value
            number2 += 1
    
    if number1 == 0:
        average_bowler_average_team1 = 10
    else:
        average_bowler_average_team1 = count1 / number1
    
    if number2 == 0:
        average_bowler_average_team2 = -10
    else:
        average_bowler_average_team2 = count2 / number2
    
    # Calculate the difference and assign to 'average_economy' column
    test_data.at[j, 'average_bowler_average'] = average_bowler_average_team1 - average_bowler_average_team2


# Creating total_innings (players) in test_data 

In [260]:
# List of column names
column_names_team1 = ['player_' + str(i) for i in range(1, 12)]
column_names_team2 = ['player2_' + str(i) for i in range(1, 12)]

# Iterate through the rows of the DataFrame
for j in range(len(test_data)):
    count1 = 0
    count2 = 0
    
    for col in column_names_team1:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count1 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count1 += corresponding_value
    
    for col in column_names_team2:
        x = test_data[col].iloc[j]
        
        # Filter the bat_csv DataFrame
        filtered_df = bat_csv[bat_csv['batsman_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count2 += corresponding_value
        
        # Filter the bow_csv DataFrame
        filtered_df = bow_csv[bow_csv['bowler_id'] == float(x)]
        if not filtered_df.empty:
            corresponding_value = filtered_df['total_innings'].iloc[0]
            count2 += corresponding_value
    
    # Calculate the difference and assign to 'star_players' column
    test_data.at[j, 'total_innings_diff'] = count1 - count2
    
    test_data

# Ground_id

In [261]:
# Merge train_data with win_df based on 'ground_id'
merged_data = pd.merge(test_data, win_df, on='ground_id', how='left')

# Fill missing win probabilities with a default value (e.g., 0) if any
merged_data['win_probability'].fillna(0, inplace=True)

# Print the merged DataFrame to verify the changes
test_data = merged_data
test_data

,match id,team1,team1_id,team2,team2_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11,star_players,average_average,average_strike_rate,average_economy,average_bowler_average,total_innings_diff,win_probability
0,9250275,Jy,28594,Ud Ss of Aa,90,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN,7438955.0,8271969.0,8369661.0,3685247.0,2590251.0,7785497.0,7785553.0,3296264.0,4613622.0,2904796.0,3685289.0,2436944.0,8117500.0,6703528.0,3010748.0,1613870.0,8934764.0,2721480.0,1632819.0,2789604.0,6489846.0,1565507.0,-6.0,9.274242,33.802843,-1.002239,-4.542612,-65.0,0.250000
1,9262189,Ga An Ws,36084,Ja Ts,36098,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.500000,3715690.0,6818622.0,4069666.0,393014.0,4690188.0,4739552.0,4690328.0,5744850.0,4739580.0,9266102.0,284339.0,4690258.0,3761246.0,8464385.0,5742470.0,3150720.0,4690104.0,181404.0,1594319.0,5509524.0,4239794.0,2036649.0,4.0,1.327733,11.286280,-0.464816,-5.857851,-129.0,0.615385
2,9128776,Rn Rs,30428,Lw Sr Gs,48334,Rn Rs,bat,Be Sm Mi,Mumbai,2022-05-15,night match,In Pr Le,2022,2764,0.842105,0.753086,171.066667,100.00,179.625000,8058959.0,2162782.0,2981614.0,7833195.0,7556051.0,2486896.0,184960.0,1945397.0,6420126.0,3011735.0,6347494.0,2654014.0,2954769.0,8058903.0,3479860.0,3299407.0,2275097.0,2740408.0,3865077.0,7924048.0,4859490.0,8228100.0,-1.0,1.221712,-5.313184,0.494276,0.296011,7.0,0.583333
3,9586919,Ja Ts,36098,St La Ks,36112,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,Gros Islet,2023-08-16,night match,Cn Pr Le,2023,11892,0.285714,1.487805,166.000000,75.00,155.500000,4690258.0,8464385.0,1613898.0,5744780.0,3150720.0,4690104.0,1594319.0,5509524.0,2036649.0,3023383.0,5471948.0,2331475.0,313809.0,391103.0,2742837.0,2097017.0,7921535.0,7459647.0,8464343.0,4690230.0,8339701.0,4702774.0,-2.0,-3.322430,-9.518144,0.179136,-0.927014,45.0,0.615385
4,9128538,Ci Sr Ks,30414,Gt Ts,48341,Gt Ts,field,Ma Ct An Sm Pe,Pune,2022-04-17,night match,In Pr Le,2022,18752,2.375000,0.310330,169.933333,0.00,164.125000,7422673.0,249087.0,62432.0,232000.0,5001170.0,1642738.0,196580.0,360086.0,2022957.0,7968225.0,7879829.0,1958683.0,7491224.0,3339160.0,5452754.0,2252452.0,2966879.0,5554254.0,4690230.0,3456424.0,3373285.0,8118053.0,0.0,1.729594,6.266735,-0.313733,-2.635788,68.0,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,9094371,Ln St,46773,Wh Fe,46752,Wh Fe,field,Ls Ln,London,2022-08-24,night match,Te Hd Ms Cn,2022,83,0.916667,2.904762,144.545455,0.00,145.181818,3200973.0,2958164.0,4223883.0,172199.0,4489974.0,1613926.0,5367676.0,1482998.0,5788418.0,2173688.0,4626236.0,8364726.0,4003390.0,2158869.0,3651472.0,2252452.0,2294823.0,4170186.0,5436878.0,3913475.0,2538983.0,2904943.0,1.0,0.218674,4.096300,-0.243038,-4.856342,82.0,0.736842
267,9440500,Si La,69,Ia,55,Ia,field,Ma Ct An Sm Pe,Pune,2023-01-05,night match,Si La tr of Ia,2022/23,18752,0.714286,1.000000,159.066667,40.00,154.950000,7200598.0,4403531.0,2398346.0,5490582.0,3260564.0,3061225.0,5490666.0,4365878.0,7968225.0,3497171.0,5551062.0,5043310.0,7491224.0,3127354.0,3125562.0,4377610.0,3479860.0,3882850.0,7558949.0,8725709.0,3011735.0,7881845.0,-5.0,-1.855950,-15.133499,-0.335785,1.3118,-189.0,0.800000
268,9085173,Sx,9967,Gn,7573,Gn,field,Sa Gs Cf,Cardiff,2022-06-19,day match,Vy Bt,2022,4521,1.888889,2.904762,174.400000,50.00,146.821429,2263736.0,164233.0,8820496.0,4215098.0,74087.0,4411231.0,63116

In [262]:
# Merge train_data with win_df based on 'ground_id'
merged_data = pd.merge(test_data, win_df_team, on='team1_id', how='left')

# Fill missing win probabilities with a default value (e.g., 0) if any
merged_data['win_probability11'].fillna(0, inplace=True)

# Print the merged DataFrame to verify the changes
test_data = merged_data
test_data

,match id,team1,team1_id,team2,team2_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player2_1,player2_2,player2_3,player2_4,player2_5,player2_6,player2_7,player2_8,player2_9,player2_10,player2_11,star_players,average_average,average_strike_rate,average_economy,average_bowler_average,total_innings_diff,win_probability,win_probability11
0,9250275,Jy,28594,Ud Ss of Aa,90,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN,7438955.0,8271969.0,8369661.0,3685247.0,2590251.0,7785497.0,7785553.0,3296264.0,4613622.0,2904796.0,3685289.0,2436944.0,8117500.0,6703528.0,3010748.0,1613870.0,8934764.0,2721480.0,1632819.0,2789604.0,6489846.0,1565507.0,-6.0,9.274242,33.802843,-1.002239,-4.542612,-65.0,0.250000,0.000000
1,9262189,Ga An Ws,36084,Ja Ts,36098,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.500000,3715690.0,6818622.0,4069666.0,393014.0,4690188.0,4739552.0,4690328.0,5744850.0,4739580.0,9266102.0,284339.0,4690258.0,3761246.0,8464385.0,5742470.0,3150720.0,4690104.0,181404.0,1594319.0,5509524.0,4239794.0,2036649.0,4.0,1.327733,11.286280,-0.464816,-5.857851,-129.0,0.615385,0.642857
2,9128776,Rn Rs,30428,Lw Sr Gs,48334,Rn Rs,bat,Be Sm Mi,Mumbai,2022-05-15,night match,In Pr Le,2022,2764,0.842105,0.753086,171.066667,100.00,179.625000,8058959.0,2162782.0,2981614.0,7833195.0,7556051.0,2486896.0,184960.0,1945397.0,6420126.0,3011735.0,6347494.0,2654014.0,2954769.0,8058903.0,3479860.0,3299407.0,2275097.0,2740408.0,3865077.0,7924048.0,4859490.0,8228100.0,-1.0,1.221712,-5.313184,0.494276,0.296011,7.0,0.583333,0.473684
3,9586919,Ja Ts,36098,St La Ks,36112,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,Gros Islet,2023-08-16,night match,Cn Pr Le,2023,11892,0.285714,1.487805,166.000000,75.00,155.500000,4690258.0,8464385.0,1613898.0,5744780.0,3150720.0,4690104.0,1594319.0,5509524.0,2036649.0,3023383.0,5471948.0,2331475.0,313809.0,391103.0,2742837.0,2097017.0,7921535.0,7459647.0,8464343.0,4690230.0,8339701.0,4702774.0,-2.0,-3.322430,-9.518144,0.179136,-0.927014,45.0,0.615385,0.533333
4,9128538,Ci Sr Ks,30414,Gt Ts,48341,Gt Ts,field,Ma Ct An Sm Pe,Pune,2022-04-17,night match,In Pr Le,2022,18752,2.375000,0.310330,169.933333,0.00,164.125000,7422673.0,249087.0,62432.0,232000.0,5001170.0,1642738.0,196580.0,360086.0,2022957.0,7968225.0,7879829.0,1958683.0,7491224.0,3339160.0,5452754.0,2252452.0,2966879.0,5554254.0,4690230.0,3456424.0,3373285.0,8118053.0,0.0,1.729594,6.266735,-0.313733,-2.635788,68.0,0.800000,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,9094371,Ln St,46773,Wh Fe,46752,Wh Fe,field,Ls Ln,London,2022-08-24,night match,Te Hd Ms Cn,2022,83,0.916667,2.904762,144.545455,0.00,145.181818,3200973.0,2958164.0,4223883.0,172199.0,4489974.0,1613926.0,5367676.0,1482998.0,5788418.0,2173688.0,4626236.0,8364726.0,4003390.0,2158869.0,3651472.0,2252452.0,2294823.0,4170186.0,5436878.0,3913475.0,2538983.0,2904943.0,1.0,0.218674,4.096300,-0.243038,-4.856342,82.0,0.736842,0.416667
267,9440500,Si La,69,Ia,55,Ia,field,Ma Ct An Sm Pe,Pune,2023-01-05,night match,Si La tr of Ia,2022/23,18752,0.714286,1.000000,159.066667,40.00,154.950000,7200598.0,4403531.0,2398346.0,5490582.0,3260564.0,3061225.0,5490666.0,4365878.0,7968225.0,3497171.0,5551062.0,5043310.0,7491224.0,3127354.0,3125562.0,4377610.0,3479860.0,3882850.0,7558949.0,8725709.0,3011735.0,7881845.0,-5.0,-1.855950,-15.133499,-0.335785,1.3118,-189.0,0.800000,0.238095
268,9085173,Sx,9967,Gn,7573,Gn,field,Sa Gs Cf,Cardiff,2022-06-19,day match,Vy Bt,2022,4521,1.888889,2.904762,174.4

In [283]:
test_data['toss_win'] = test_data.apply(lambda row: 1 if row['team1'] == row['toss winner'] else 0, axis=1)
# List of columns to keep
columns_to_keep = ['team_count_50runs_last15','team_winp_last5','team1only_avg_runs_last15','team1_winp_team2_last15','star_players','average_average','average_economy','average_bowler_average','average_strike_rate','total_innings_diff','toss_win','win_probability','win_probability11']
# Create a new DataFrame with only the specified columns
test_df = test_data[columns_to_keep]

test_df['average_economy'] = pd.to_numeric(test_df['average_economy'], errors='coerce')
test_df['average_bowler_average'] = pd.to_numeric(test_df['average_bowler_average'], errors='coerce')
test_df

C:\Users\Chitwan\AppData\Local\Temp\ipykernel_3200\2960563383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['average_economy'] = pd.to_numeric(test_df['average_economy'], errors='coerce')
C:\Users\Chitwan\AppData\Local\Temp\ipykernel_3200\2960563383.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['average_bowler_average'] = pd.to_numeric(test_df['average_bowler_average'], errors='coerce')


,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,star_players,average_average,average_economy,average_bowler_average,average_strike_rate,total_innings_diff,toss_win,win_probability,win_probability11
0,0.000000,0.019608,NaN,0.00,-6.0,9.274242,-1.002239,-4.542612,33.802843,-65.0,0,0.250000,0.000000
1,0.615385,0.344262,151.285714,66.67,4.0,1.327733,-0.464816,-5.857851,11.286280,-129.0,0,0.615385,0.642857
2,0.842105,0.753086,171.066667,100.00,-1.0,1.221712,0.494276,0.296011,-5.313184,7.0,1,0.583333,0.473684
3,0.285714,1.487805,166.000000,75.00,-2.0,-3.322430,0.179136,-0.927014,-9.518144,45.0,0,0.615385,0.533333
4,2.375000,0.310330,169.933333,0.00,0.0,1.729594,-0.313733,-2.635788,6.266735,68.0,0,0.800000,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,0.916667,2.904762,144.545455,0.00,1.0,0.218674,-0.243038,-4.856342,4.096300,82.0,0,0.736842,0.416667
267,0.714286,1.000000,159.066667,40.00,-5.0,-1.855950,-0.335785,1.311800,-15.133499,-189.0,0,0.800000,0.238095
268,1.888889,2.904762,174.400000,50.00,3.0,4.551626,-0.652571,-17.152282,3.455261,370.0,0,0.600000,0.375000
269,0.333333,1.487805,171.066667,100.00,4.0,0.945037,0.390624,2.056932,9.379946,182.0,0,0.526316,0.388889


In [284]:
# Assuming model is already trained and test_df is your new data DataFrame

# Predict the outcomes
y_pred_new = model.predict(test_df)
y_pred_prob_new = model.predict_proba(test_df)[:, 1]
y_pred_binary_new = [1 if x > 0.5 else 0 for x in y_pred_prob_new]
max_values = [max(x, 1 - x) for x in y_pred_prob_new]

print(y_pred_binary_new)


[0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0]


In [285]:
merged_data = pd.read_csv(r"C:\Users\Chitwan\Downloads\merged_data (1).csv")

# Compare the combined predictions with the actual winner column
matches = sum(y_pred_binary_new== merged_data['winner'])

print(f"Number of times combined prediction is equal to winner column: {matches}")

Number of times combined prediction is equal to winner column: 174


In [77]:
max_values 

[0.5164145321287024,
 0.5139088176914571,
 0.5006957552718476,
 0.5018385673736702,
 0.5067748162143619,
 0.5048120210011873,
 0.5115332896293768,
 0.5184803590171034,
 0.5077418463203296,
 0.5034482770494357,
 0.506300515370346,
 0.5159892958003935,
 0.5047364598890731,
 0.504956064339254,
 0.5092942792658683,
 0.5001154648264649,
 0.5054803278119909,
 0.5010676043765953,
 0.5189538745264798,
 0.5021801623933669,
 0.507087269155335,
 0.5139088176914571,
 0.5072939423556209,
 0.5147308207312891,
 0.512956895995182,
 0.5115332896293768,
 0.5210250092782204,
 0.5158272886956826,
 0.5181774784327534,
 0.5159892958003935,
 0.5006076143245489,
 0.5080559238750156,
 0.5091095896871326,
 0.5060293088351457,
 0.5151286394087092,
 0.5190863163075329,
 0.5061303806834961,
 0.5038926323337208,
 0.5039411653785283,
 0.5088516441456153,
 0.5052061740307716,
 0.5025894098834098,
 0.505120489809909,
 0.5136165596526308,
 0.5085738071560154,
 0.5111004467322007,
 0.5149067031637884,
 0.516636388888418

In [78]:
from collections import Counter
count_result = Counter(y_pred_binary_new)
count_result

Counter({0: 152, 1: 119})

In [80]:
import pandas as pd

# Assuming test_data contains the data as described
# Assuming y_pred_binary_new is the vector of predictions

# Create an empty list to store the team IDs of the predicted winners
q_team_ids = []

# Iterate through the rows of test_data
for index, row in test_data.iterrows():
    if y_pred_binary_new[index] == 1:
        # If the prediction is 1, add team1_id to q_team_ids
        q_team_ids.append(row['team1_id'])
    else:
        # If the prediction is 0, add team2_id to q_team_ids
        q_team_ids.append(row['team2_id'])

# Convert q_team_ids to a numpy array if needed
q_team_ids = np.array(q_team_ids)

# Now you have the team IDs corresponding to the predictions in y_pred_binary_new
csv_file_path = "sub23.csv"

# Save the NumPy array to a CSV file
np.savetxt(csv_file_path, q_team_ids, delimiter=",", fmt='%d')

In [ ]:
csv_file_path = "sub6_probability.csv"

# Save the NumPy array to a CSV file
np.savetxt(csv_file_path, max_values , delimiter=",", fmt='%f')